In [37]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [38]:
data = pd.read_csv(r'/home/arnavgupta829/Desktop/ELL409/Assignment2/OldFaithful.csv')
raw_ = []
for i in data.columns:
    raw_.append(data[i])
raw = np.asarray(raw_)
raw = raw.transpose()

In [39]:
def computeDistance(a, b):
    return np.linalg.norm(a-b)

def distBetweenClusters(u, v):
    distances = []
    for i in u.rep:
        for j in v.rep:
            distances.append(computeDistance(i, j))
    return np.amin(distances)

In [40]:
class Cluster:
    
    def __init__(self):
        self.allPoints = []
        self.mean = None
        self.rep = []
        self.closest = None

In [41]:
class HeapNode:
    def __init__(self, key, cluster):
        self.key = key
        self.cluster = cluster

In [42]:
class MinHeap:
    def __init__(self):
        self.storedLength = 0
        self.heap = []
        self.heap.append(HeapNode((-1*sys.maxsize), None))
        
    def parent(self, pos):
        return pos//2
    
    def leftChild(self, pos):
        return (2*pos)
    
    def rightChild(self, pos):
        return (2*pos)+1
    
    def isLeaf(self, pos):
        if(pos>= (self.size//2) and pos<=self.size):
            return true
        return false
    
    def swap(self, fpos, spos):
        self.heap[fpos], self.heap[spos] = self.heap[spos], self.heap[fpos]
    
    def heapify(self, pos):
        if(not self.isLeaf(pos)):
            if(self.heap[pos].key > self.heap[self.leftChild[pos]].key or self.heap[pos].key > self.heap[self.rightChild[pos]].key):
                if(self.heap[self.leftChild[pos]].key < self.heap[self.rightChild[pos]].key):
                    self.swap(pos, self.leftChild(pos))
                    self.heapify(self.leftChild(pos))   
                else: 
                    self.swap(pos, self.rightChild(pos))
                    self.heapify(self.rightChild(pos))
    
    def checkHeap(self):
        for pos in range(self.size//2, 0, -1):
            self.heapify(pos)
    
    def insert(self, key, cluster):
        toInsert = HeapNode(key, cluster)
        self.storedLength += 1
        if(self.storedLength < len(self.heap)):
            heap[self.storedLength] = toInsert
        else:
            heap.append(toInsert)
        current = self.storedLength
        while(self.heap[current] < self.heap[self.parent(current)]):
            self.swap(current, self.parent(current))
            current = self.parent

#Need to finish delete, extractMin

In [43]:
class Node:
    def __init__(self, key, cluster):
        self.parent = None
        self.left = None
        self.right = None
        self.key = key
        self.cluster = cluster

In [44]:
class KDTree_2_:
    def __init__(self):
        self.root = None
    
    def recInsert(self, parent, node, toAdd, depth):
        if(node == None):
            toAdd.parent = parent
            node = toAdd
            return node
        plane = depth%2
        if(key[plane] < node.key[plane]):
            node.left = recInsert(node, node.left, toAdd, depth+1)
        else:
            node.right = recInsert(node, node.right, toAdd, depth+1)
        return node
    
    def insertKey(self, key, cluster):
        toAdd = Node(key, cluster)
        recInsert(None, self.root, toAdd, 0)
    
    def minNode(self, node1, node2, node3, plane):
        min = node1
        if(node2 != None and (min.key[plane] > node2.key[plane])):
            min = node2
        if(node3 != None and (min.key[plane] > node3.key[plane])):
            min = node3
        return min
    
    def recFindMin(self, node, d, depth):
        if(node == None):
            return None
        plane = depth%2
        if(d == plane):
            if(node.left == None):
                return node
            return recFindMin(node.left, d, depth+1)
        return minNode(node, recFindMin(node.left, d, depth+1), recFindMin(node.right, d, depth+1))
    
    def findMin(self, node, d):
        return recFindMin(node, d, 0)
    
    def recDelete(self, node, key, depth):
        if(node == None):
            return None
        plane = depth%2
        if(node.key == key):
            if(node.right != None):
                min = findMin(node.right, plane)
                node.key = min.key
                node.cluster = min.cluster
                node.right = recDelete(node.right, min.key, depth+1)
            elif(node.left != None):
                min = findMin(node.left, plane)
                node.key = min.key
                node.cluster = min.cluster
                node.left = recDelete(node.left, min.key, depth+1)
            else:
                node = None
                return node
            return node
        if(key[plane] < node.key[plane]):
            node.left = recDelete(node.left, key, depth+1)
        else:
            node.right = recDelete(node.right, key, depth+1)
        return node
        
    def deleteKey(self, key):
        root = recDelete(self.root, key, 0)

In [45]:
def merge(u, v, c, a):
    w = Cluster()
    for i in u.allPoints:
        w.allPoints.append(i)
    for j in v.allPoints:
        w.allPoints.append(j)
    w.mean = (((len(u.allPoints)*u.mean)+(len(v.allPoints)*v.mean))/(len(u.allPoints)+len(v.allPoints)))
    rep_ = []
    for i in range(0, c):
        maxDist = 0
        for j in w.allPoints:
            if(i == 0):
                minDist = computeDistance(j, w.mean)
            else:
                distances = []
                for k in rep_:
                    distances.append(computeDistance(j, k))
                minDist = np.amin(distances)
            if(minDist>maxDist):
                maxDist = minDist
                maxPoint = j
        rep_.append(maxPoint)
    for i in rep_:
        w.rep.append(i+(a*(w.mean-i)))
    return w

In [46]:
class CURE:
    def __init__(k, datapoints):
        self.k = k

#complete the clustering algorithm